In [13]:
import numpy as np
import pandas as pd
from river import evaluate
from sail.models.river.forest import AdaptiveRandomForestRegressor
from sail.pipeline import SAILPipeline
from sklearn.impute import SimpleImputer
from sail.transformers.river.preprocessing import StandardScaler

#### Load Data


In [14]:
X = pd.read_csv("../datasets/HDWF2.csv").head(2000)

y = X["power"]
X.drop(["power", "time"], axis=1, inplace=True)

#### Model Definition


In [15]:
random_forest = AdaptiveRandomForestRegressor()

#### Create SAIL Pipeline


In [16]:
steps = [
    ("Imputer", SimpleImputer(missing_values=np.nan, strategy="mean")),
    ("standard_scalar", StandardScaler()),
    ("classifier", random_forest),
]
sail_pipeline = SAILPipeline(steps=steps, scoring="R2", verbosity_level=1, verbosity_interval=2)

#### Train Test Split


In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

#### Start Incremental Training


In [18]:
y_preds = []
y_true = []
batch_size = 100

for start in range(0, 1000, batch_size):

    end = start + batch_size

    X = X_train.iloc[start:end]
    y = y_train.iloc[start:end]

    sail_pipeline.partial_fit(X, y)


>>> Epoch: 1 | Samples Seen: 0 -------------------------------------------------------------------------------------


SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.132s, Batch Size=100, P_Score=0]             



>>> Epoch: 2 | Samples Seen: 100 -------------------------------------------------------------------------------------


SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.143s, Batch Size=100, P_Score=0.816]             



>>> Epoch: 4 | Samples Seen: 300 -------------------------------------------------------------------------------------


...

SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.149s, Batch Size=100, P_Score=0.806]             



>>> Epoch: 8 | Samples Seen: 700 -------------------------------------------------------------------------------------


SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.143s, Batch Size=100, P_Score=0.797]             



>>> Epoch: 10 | Samples Seen: 900 -------------------------------------------------------------------------------------


SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.135s, Batch Size=100, P_Score=0.804]             


In [19]:
sail_pipeline._scorer

#### Save SAIL Pipeline


In [7]:
sail_pipeline.save(".")

'./sail_pipeline'

#### Load SAIL Pipeline


In [8]:
sail_new_pipeline = SAILPipeline.load(".")

SAIL Pipeline Progressive Score: 100%========== [Points: 900/900, Elapsed:0.0027s, Metric=R2, Batch Size=900, P_Score=0.782]


In [9]:
y_preds = []
y_true = []
batch_size = 100

for start in range(1001, 1400, batch_size):

    end = start + batch_size

    X = X_train.iloc[start:end]
    y = y_train.iloc[start:end]

    sail_new_pipeline.partial_fit(X, y)


>>> Epoch: 12 | Samples Seen: 1100 -------------------------------------------------------------------------------------


SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.148s, Batch Size=100, P_Score=0.792]             



>>> Epoch: 14 | Samples Seen: 1300 -------------------------------------------------------------------------------------


SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.142s, Batch Size=99, P_Score=0.8]             


#### Make Prediction on Hold out set


In [10]:
y_preds = []
y_true = []
batch_size = 100

for start in range(0, X_test.shape[0], batch_size):

    end = start + batch_size

    X = X_test.iloc[start:end]
    y = y_test.iloc[start:end]

    preds = sail_new_pipeline.predict(X)
    y_preds.extend(list(preds))
    y_true.extend(list(y))

#### Final Score


In [11]:
sail_new_pipeline.get_progressive_score

0.8001829714616957

In [12]:
import plotly.express as px

df = pd.DataFrame({"y_true": y_true, "y_preds": y_preds}).head(100)
fig = px.line(df, y=["y_true", "y_preds"], title='')
fig.show()